In [4]:
from randdiag import *
from time import time

In [9]:
def offdiagonal_frobenius(A):
    loss = np.linalg.norm(A - np.diag(np.diagonal(A)),'fro')
    return loss

def compute_distance_eigenvalue_unitary(D1,D2):
    s = 0
    for d in D1:
        s+= np.min(np.absolute(D2-d))
    return s / D1.size

def compare_algorithms(ns = [10,100,1000],repeats = 100):
    for n in ns:
        print('Matrix size: ', n)
        A = np.random.randn(n,n) + 1j*np.random.randn(n,n)
        U,_ = np.linalg.qr(A)
        rt_rjd = 0
        err_rjd = 0
        for _ in range(repeats):
            start = time()
            Q = randdiag(U)
            rt_rjd += time()-start
            err_rjd += offdiagonal_frobenius(Q.conj().T @ U @ Q)
        print("RandDiag: {:.2f}, {:.2e}".format( rt_rjd / repeats, err_rjd / repeats))
        rt_schur = 0
        err_schur = 0
        for _ in range(repeats):
            start = time()
            T,Z = schur(U, 'complex')
            rt_schur+=time()-start
            err_schur += offdiagonal_frobenius(Z.conj().T @ U @ Z)
            #print(np.linalg.norm(Z.conj().T @ Z -np.eye(n)))
        print("SCHUR: {:.2f}, {:.2e}".format( rt_schur / repeats, err_schur / repeats))
        rt_schur = 0
        for _ in range(repeats):
            start = time()
            D = eigvals(U)
            rt_schur+=time()-start
        print("Schur Eigenvalue only: {:.2f}".format( rt_schur / repeats))
        rt_hess = 0
        for _ in range(repeats):
            start = time()
            D = eigenvalue_unitary_angle(U)
            rt_hess+=time()-start
        print("Rand Eigval: {:.2f}".format( rt_hess / repeats))

In [10]:
compare_algorithms(ns = [100],repeats=1)

Matrix size:  100
RandDiag: 0.01, 8.23e-13
SCHUR: 0.02, 4.61e-14
Schur Eigenvalue only: 0.00
Rand Eigval: 0.02


In [17]:
def compare_algorithms_eigenvalue(ns = [10,100,1000],repeats = 100):
    for n in ns:
        print('Matrix size: ', n)
        A = np.random.randn(n,n) + 1j*np.random.randn(n,n)
        U,_ = np.linalg.qr(A)
        rt_schur = 0
        err_schur = 0
        D_true = eigvals(U)
        rt_rjd = 0
        err_rjd = 0
        for _ in range(repeats):
            start = time()
            Q = randdiag(U)
            D = np.diag(Q.conj().T @ U @ Q)
            rt_rjd += time()-start
            err_rjd += compute_distance_eigenvalue_unitary(D,D_true)
        print("RandDiag: {:.2f}, {:.2e}".format( rt_rjd / repeats, err_rjd / repeats))
        for _ in range(repeats):
            start = time()
            D = eigvals(U)
            rt_schur+=time()-start
            err_schur += compute_distance_eigenvalue_unitary(D ,D_true)
        print("SCHUR: {:.2f}, {:.2e}".format( rt_schur / repeats, err_schur / repeats))
        rt_eigval = 0
        err_eigval = 0
        for _ in range(repeats):
            start = time()
            D = eigenvalue_unitary_angle(U)
            rt_eigval+=time()-start
            err_eigval+= compute_distance_eigenvalue_unitary(np.exp(1j*D),D_true)
        print("Rand Eigval: {:.2f}, {:.2e}".format( rt_eigval / repeats,err_eigval / repeats))
compare_algorithms_eigenvalue([1000],100)

Matrix size:  1000
RandDiag: 0.63, 4.36e-15
SCHUR: 1.98, 0.00e+00
Rand Eigval: 0.61, 7.08e-15


In [15]:
n = 1000
A = np.random.randn(n,n)
U,_ = np.linalg.qr(A)
repeats = 1

In [137]:
rt_rjd = 0
err_rjd = 0
for _ in range(repeats):
    start = time()
    H = (U+U.conj().T) / 2; S = (U-U.conj().T) / 2
    AA = np.array([H,1j*S])
    mu = np.random.normal(0,1,2)
    A_mu = np.einsum('ijk,i->jk',AA, mu)
    _, Q = np.linalg.eigh(A_mu)
    rt_rjd += time()-start
    err_rjd += offdiagonal_frobenius(Q.conj().T @ U @ Q)
print(rt_rjd / repeats, err_rjd / repeats)

0.6310639381408691 1.6923075836340513e-11


In [138]:
rt_schur = 0
err_schur = 0
for _ in range(repeats):
    start = time()
    T,Z = schur(U, 'complex')
    rt_schur+=time()-start
    err_schur += offdiagonal_frobenius(T)
print(rt_schur / repeats, err_schur / repeats)

2.3382792472839355 3.7858406276739907e-13


In [213]:
n = 10
A = np.random.randn(n,n) #+ 1j*np.random.randn(n,n)
U,_ = np.linalg.qr(A)
start = time()
H = (U+U.conj().T) / 2; S = (U-U.conj().T) / 2
mu = np.random.normal(0,1,2)
#mu[0] = np.abs(mu[0])
#mu[1] = -np.abs(mu[1])
A_mu = mu[0] * H + mu[1] * 1j*S #+ mu[2]*1j * H + mu[3]* 1j * 1j * S
D1 = eigvalsh(A_mu)
D2 = eigvalsh(H)
D2 = np.arccos(np.clip(D2,-1,1))
D2 = np.concatenate([D2,-D2])

angle = np.angle( mu[0]-1j*mu[1])
radius = np.absolute(mu[0]-1j*mu[1])
D1 = D1 / radius
D1 = np.arccos(D1)
D1_plus = angle+D1; D1_plus = D1_plus + (D1_plus > np.pi) * (- 2*np.pi) 
D1_minus = angle-D1; D1_minus = D1_minus  + (D1_minus < -np.pi) * (2*np.pi)

condition = np.array([ True if np.min(np.abs(D1_plus[x] - D2)) < np.min(np.abs(D1_minus[x] - D2)) \
                      else False for x in range(D1.size)])
D1 = np.where(condition,D1_plus,D1_minus)

D0 = eigvals(U)
print('D2: ', D2)
print('D1 plus: ', D1_plus )
print('D1 minus: ', D1_minus )
print('true angle:', np.sort(np.angle(D0)))
print('Result: ', np.sort(D1))
print(np.linalg.norm(np.sort(D1)- np.sort(np.angle(D0))))

D2:  [ 3.14159262  2.29246813  2.29246813  2.13070154  2.13070154  1.54961961
  1.54961961  1.12084268  1.12084268  0.         -3.14159262 -2.29246813
 -2.29246813 -2.13070154 -2.13070154 -1.54961961 -1.54961961 -1.12084268
 -1.12084268 -0.        ]
D1 plus:  [-1.12084268 -1.35171081 -1.54961961 -2.13070154 -2.29246813 -2.47255349
 -2.90133041 -3.14159265  2.80077296  2.63900637]
D1 minus:  [-0.23086813  0.          0.1979088   0.77899073  0.94075732  1.12084268
  1.54961961  1.78988184  2.13070154  2.29246813]
true angle: [-2.29246813 -2.13070154 -1.54961961 -1.12084268  0.          1.12084268
  1.54961961  2.13070154  2.29246813  3.14159265]
Result:  [-3.14159265 -2.29246813 -2.13070154 -1.54961961 -1.12084268  0.
  1.12084268  1.54961961  2.13070154  2.29246813]
2.247209411142166


In [188]:
print(eigenvalue_unitary_angle(U).size)

100
